In [1]:
# Source config and checkpoints
#checkpoint_path = "gs://patemotter-maxtext-out/patemotter-1b-maxtext-model/checkpoints/3399"
#config_path = "gs://patemotter-maxtext-out/patemotter-1b-maxtext-model/base.yml"

# Copied files into our own bucket
checkpoint_path = "gs://patflick-maxtext-lingvo/patemotter-1b/checkpoints/3399"
config_path = "gs://patflick-maxtext-lingvo/patemotter-1b/base.yml"
config_path = "config/base-1b.yml"

In [2]:
!pwd

/home/patflick/dev/git/maxtext/MaxText


In [3]:
my_bucket = "gs://patflick-maxtext-lingvo"
base_output_directory= my_bucket + "/maxtext/1b/2023114"

checkpoint_path = "gs://patflick-maxtext-lingvo/patemotter-1b/checkpoints"
extra_config = {
    "run_name": "finetuning-1b-v1",
    "base_output_directory": base_output_directory,
    "load_from_other_directory": checkpoint_path,
    "load_from_other_directory_step": 3399,
    "file_pattern_for_train_data": "gs://unused", # TODO: fix upstream
    "prompt": "What can you tell me about the performance of TPUs?",
    "max_predict_length": 300,
    "dcn_tensor_parallelism": 4,
    "ici_tensor_parallelism": 4,

    # Downloaded from 
    "dataset_path": my_bucket + "/",
    "assets_path": "/home/patflick/dev/git/maxtext/assets", # TODO: move assets to GCS
    #"dataset_path": #"gs://maxtext-dataset/",
}

import pyconfig

def get_config(config_file, extra_config):
    args = ["", config_file]
    for k, v in extra_config.items():
        args.append(f"{k}={v}")
    pyconfig.initialize(args)

    return pyconfig.config

config = get_config("configs/base-1b.yml", extra_config)
pyconfig._config.keys

OrderedDict([('run_name', 'finetuning-1b-v1'),
             ('load_parameters_path', ''),
             ('load_from_other_directory',
              'gs://patflick-maxtext-lingvo/patemotter-1b/checkpoints'),
             ('load_from_other_directory_step', 3399),
             ('reuse_example_batch', 0),
             ('metrics_file', ''),
             ('gcs_metrics', False),
             ('dtype', dtype(bfloat16)),
             ('int8_training', False),
             ('use_dqdg', False),
             ('fwd_int8', True),
             ('dlhs_int8', True),
             ('drhs_int8', False),
             ('fwd_int8_qk', False),
             ('dlhs_int8_qk', False),
             ('drhs_int8_qk', False),
             ('fwd_int8_pv', True),
             ('dlhs_int8_pv', True),
             ('drhs_int8_pv', False),
             ('aqt_use_dummy_static_bound', False),
             ('aqt_use_fwd_quant', False),
             ('aqt_rng_type', 'jax.uniform'),
             ('global_parameter_scale', 1),
 

In [4]:
import decode
import os
import jax

os.environ["TFDS_DATA_DIR"] = pyconfig.config.dataset_path + "/"
decode.decode_loop(config) 
jax.distributed.shutdown()

2023-11-15 20:06:57.282117: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/patflick/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Initialized persistent compilation cache at /home/patflick/jax_cache


Creating checkpoint manager...
Checkpoint manager created!
Devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)] (num_devices: 4)
Decided on mesh: [[[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)
   TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0)
   TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0)
   TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)]]]
Model path: /home/patflick/dev/git/maxtext/assets/tokenizer
restoring state from gs://patflick-maxtext-lingvo/patemotter-1b/checkpoints step 3399


I0000 00:00:1700078822.910023  302588 gcs_resource.cc:99] Using default AdmissionQueue with limit 32
I0000 00:00:1700078822.913320  305197 google_auth_provider.cc:179] Running on GCE, using service account 903354779218-compute@developer.gserviceaccount.com


Decoding #0 (num tokens 300):
	What can you tell me about the performance of TPUs? how much of the TSPysysysysysysyssyssyssssysssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss
Decoding #1 (num tokens 300):
	What can you tell me about the performance of TPUs? is not on my a . I don's , I's abt i's that i can i can's .m not bet's. i't's. i't't arrr's. i's not. i's i's i't's i's i's . is, i's not i's i's. i's i's i's. i't's.'s i's i's . i's i's i's i's i's i's . i's i's .'s i's i's i's. i's i's i's i's's.'s i's i's i's's i's i's. i's .'s i's i's .s.s i's i's i's i's i's . is's i's i's i's i's i's i's i's i's i's i's i's i's i's i's i's i's i's i's i
Decoding #2 (num tokens 300):
	What can you tell me about the performance of TPUs? i'ss in .s in the ...i.i.. i's.i.i.i.e.i.i.i.i.i.i.i.i.i

KeyboardInterrupt: 

In [ ]:
# 
import jax
jax.distributed.shutdown()